In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import pandas as pd
import numpy as np

In [3]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [4]:
# ID of your customer here
# CUSTOMER_SERVICE_ID = '5922380045'
CUSTOMER_SERVICE_ID = '3637290511'
# Load customer account access
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [ ]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
        if 'enumValues' in field:
            print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

In [ ]:
campaign_id = 1747836661

In [ ]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=campaign_id)


In [ ]:
campaign_insights.info()
campaign_insights

In [ ]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=campaign_id)

In [ ]:
adgroup_insights.info()
adgroup_insights

In [ ]:
AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'Id', 'AdGroupStatus',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_age_range_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_age_range_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AGE_RANGE_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'AGE_RANGE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AGE_RANGE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AGE_RANGE_FIELDS)),
            columns=AGE_RANGE_FIELDS
        )
        df = df.append(df_temp)
#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

# if __name__ == '__main__':
# Initialize client object.
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_age_range_insights(client)
adgroup_insights = get_age_range_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [ ]:
adgroup_insights

In [ ]:
AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'AdGroupStatus',
                   'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion',
                   'DisplayName'
                  ]
TEST_FIELDS = ['DisplayName']
# AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_audience_insights(client, campaign_id=None, adgroup_id=None, date_preset=None, performance_type='Url'):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_audience_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': performance_type+'_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': performance_type+'_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AUDIENCE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    import csv    
    with open(performance_type+'.csv', 'wb') as output_file:
        report_downloader.DownloadReport( 
            report, output=output_file, skip_report_header=True, skip_column_header=False,
            skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    with open(performance_type+'.csv')as csv_file:
        df = pd.read_csv(csv_file,sep=",", quotechar='"')
        return df

#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

#     return df.reset_index(drop=True)

# if __name__ == '__main__':
#     # Initialize client object.
#     client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
# #     adgroup_insights = get_audience_insights(client)
#     adgroup_insights = get_audience_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [ ]:
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adgroup_insights = get_audience_insights(client, campaign_id=1747836670, adgroup_id=68524904775, performance_type='PLACEMENT')

In [ ]:
adgroup_insights[adgroup_insights.Clicks != 0].sort_values(by=['Clicks'], ascending=False)

In [ ]:
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.
operations = [{
    'operator': 'SET',
    'operand': {
        'id': ad_group_id,
        'status': 'ACTIVE'
    }
}]

if bid_micro_amount:
    operations[0]['operand']['biddingStrategyConfiguration'] = {
      'bids': [{
          'xsi_type': 'CpcBid',
          'bid': {
              'microAmount': bid_micro_amount,
          }
      }]
    }

ad_groups = ad_group_service.mutate(operations)

In [ ]:
from googleads import adwords
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
AD_GROUP_ID = 68524904055
CPC_BID_MICRO_AMOUNT = '4000000'


def update_adgroup_bid(client, ad_group_id, bid_micro_amount=None):
    # Initialize appropriate service.
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    
    # Construct operations and update an ad group.
    operations = [{
        'operator': 'SET',
        'operand': {
            'id': ad_group_id,
            'status': 'ENABLED'
        }
    }]
  
    if bid_micro_amount:
        operations[0]['operand']['biddingStrategyConfiguration'] = {
            'bids': [{
                'xsi_type': 'CpcBid',
                'bid': {
                    'microAmount': bid_micro_amount,
                }
            }]
        }
  
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [ ]:

adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [28]:
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
def retrieve_ad_group_params(client, ad_group_id):
    _FIELDS = ['AdGroupId', 'CriteriaType', 'UserInterestId', 'UserInterestName', 'UserListId', 'LabelIds']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_group_service = client.GetService('AdGroupCriterionService', version='v201809')
    ad_group_params = ad_group_service.get(selector)
    return ad_group_params


In [27]:
import copy
###
new_adgroup_id = 67966950646
###
def make_adgroup_criterion(new_adgroup_id = None, old_ad_group_params=None):
    if new_adgroup_id is None and old_ad_group_params is None:
        raise ValueError('new_adgroup_id and old_ad_group_params is Required.')
    biddable_criterion['adGroupId'] = new_adgroup_id
    negative_criterion['adGroupId'] = new_adgroup_id

    biddable_ad_group_criteria_list = list()
    negative_ad_group_criteria_list = list()
    for entry in old_ad_group_params['entries']:
        criterion = dict()

        if 'AgeRange' in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['Criterion.Type'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']

        elif 'Placement'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['url'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']
            criterion['url'] = entry['criterion']['url']

        elif 'Keyword'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['text'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']
            criterion['matchType'] = entry['criterion']['matchType']
            criterion['text'] = entry['criterion']['text']
    #         print(criterion)

        elif 'CriterionUserInterest'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['userInterestId'] = entry['criterion']['userInterestId']

        if criterion:
            if 'BiddableAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                biddable_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                biddable_criterion['criterion'] = criterion
                biddable_ad_group_criteria_list.append(copy.deepcopy(biddable_criterion))

            elif 'NegativeAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                negative_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                negative_criterion['criterion'] = criterion
                negative_ad_group_criteria_list.append(copy.deepcopy(negative_criterion))
    return biddable_ad_group_criteria_list, negative_ad_group_criteria_list
 

In [26]:
def assign_criterion_to_ad_group(client, ad_group_id, ad_group_params):
    '''
    an example of addong 4 type of criterion to new adgroup,
    if trying to exclude certain criterions, 
    change 'xsi_type' from 'BiddableAdGroupCriterion' to 'NegativeAdGroupCriterion' in ad_group_criteria object
    '''
    # Initialize appropriate service.
    ad_group_criterion_service = client.GetService(
        'AdGroupCriterionService', version='v201809')
    # Create the ad group criteria.
    ad_group_criteria = ad_group_params
  
    # Create operations.
    operations = []
    for criterion in ad_group_criteria:
        operations.append({
            'operator': 'ADD',
            'operand': criterion
        })
  
    response = ad_group_criterion_service.mutate(operations)
    return response

In [20]:
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and add ad groups.
operations = [{
    'operator': 'ADD',
    'operand': {
        'campaignId': 1777436620,
        'name': 'TEST_2',
        'status': 'PAUSED',
        'biddingStrategyConfiguration': {
            'bids': [
                {
                    'xsi_type': 'CpcBid',
                    'bid': {
                        'microAmount': '1000000'
                    },
                }
            ]
        },
    }
}]
ad_groups = ad_group_service.mutate(operations)

In [25]:
new_ad_group_id = ad_groups['value'][0]['id']

In [30]:
old_ad_group_params = retrieve_ad_group_params(client, 67966950646)
biddable_params, negative_params = make_adgroup_criterion(new_ad_group_id, old_ad_group_params)
new_ad_group_params = biddable_params + negative_params
result = assign_criterion_to_ad_group(client, new_ad_group_id, new_ad_group_params)

In [31]:
result

{
    'ListReturnValue.Type': 'AdGroupCriterionReturnValue',
    'value': [
        {
            'adGroupId': 69550439176,
            'criterionUse': None,
            'criterion': {
                'id': 503002,
                'type': 'AGE_RANGE',
                'Criterion.Type': 'AgeRange',
                'ageRangeType': 'AGE_RANGE_25_34'
            },
            'labels': [],
            'forwardCompatibilityMap': [],
            'baseCampaignId': None,
            'baseAdGroupId': None,
            'AdGroupCriterion.Type': 'BiddableAdGroupCriterion',
            'userStatus': 'ENABLED',
            'systemServingStatus': 'ELIGIBLE',
            'approvalStatus': 'APPROVED',
            'disapprovalReasons': [],
            'firstPageCpc': None,
            'topOfPageCpc': None,
            'firstPositionCpc': None,
            'qualityInfo': None,
            'biddingStrategyConfiguration': {
                'biddingStrategyId': None,
                'biddingStrategyName': N

In [32]:
retrieve_ad_group_params(client, new_ad_group_id)

{
    'totalNumEntries': 19,
    'Page.Type': 'AdGroupCriterionPage',
    'entries': [
        {
            'adGroupId': 69550439176,
            'criterionUse': 'NEGATIVE',
            'criterion': {
                'id': 503001,
                'type': 'AGE_RANGE',
                'Criterion.Type': 'AgeRange',
                'ageRangeType': 'AGE_RANGE_18_24'
            },
            'labels': [],
            'forwardCompatibilityMap': [],
            'baseCampaignId': None,
            'baseAdGroupId': None,
            'AdGroupCriterion.Type': 'NegativeAdGroupCriterion'
        },
        {
            'adGroupId': 69550439176,
            'criterionUse': 'BIDDABLE',
            'criterion': {
                'id': 503002,
                'type': 'AGE_RANGE',
                'Criterion.Type': 'AgeRange',
                'ageRangeType': 'AGE_RANGE_25_34'
            },
            'labels': [],
            'forwardCompatibilityMap': [],
            'baseCampaignId': None,
       